In [3]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [19]:
df_train = pd.read_csv(r"data/Poem_classification - train_data.csv")
df_test = pd.read_csv(r"data/Poem_classification - test_data.csv")

In [22]:
df_train = df_train.dropna(axis=0)
df_train.isnull().sum()

Genre    0
Poem     0
dtype: int64

In [23]:
df_test.isnull().sum()

Genre    0
Poem     0
dtype: int64

In [25]:
display(df_train)
display(df_test)

,Genre,Poem
1,Music,In the thick brushthey spend the...
2,Music,Storms are generous. ...
3,Music,—After Ana Mendieta Did you carry around the ...
4,Music,for Aja Sherrard at 20The portent may itself ...
5,Music,"for Bob Marley, Bavaria, November 1980 Here i..."
...,...,...
836,Environment,Why make so much of fragmentary blue In here a...
837,Environment,"Woman, I wish I didn't know your name. What co..."
838,Environment,"Yonder to the kiosk, beside the creek, Paddle ..."
839,Environment,You come to fetch me from my work to-night Whe...


,Genre,Poem
0,Music,A woman walks by the bench I’m sitting onwith ...
1,Music,"Because I am a boy, the untouchability of beau..."
2,Music,"Because today we did not leave this world,We n..."
3,Music,"Big Bend has been here, been here. Shouldn’t i..."
4,Music,"I put shells there, along the lip of the road...."
...,...,...
145,Environment,"To pick a tulip from the garden, the red one. ..."
146,Environment,We are as clouds that veil the midnight moon; ...
147,Environment,"When pulled, the spider web took another form...."
148,Environment,Whose woods these are I think I know. His hous...


In [27]:
df_train.Genre.value_counts()


Music          238
Death          231
Environment    227
Affection      141
Name: Genre, dtype: int64

In [28]:
df_test.Genre.value_counts()

Affection      100
Environment     25
Death           13
Music           12
Name: Genre, dtype: int64